In [ ]:
import os
import pandas as pd
from difflib import SequenceMatcher

# Cargar los archivos de Excel
file_path_1 = 'Archivo1.xlsx'  # Archivo de la base
file_path_2 = 'Archivo2.xlsx'  # Archivo de solicitud

# Leer las hojas de cálculo de ambos archivos
sheet1_df = pd.read_excel(file_path_1, sheet_name=0)  # Base de datos
sheet2_df = pd.read_excel(file_path_2, sheet_name=0)  # Solicitud de comparación

# Mostrar las primeras filas de ambos archivos para entender su estructura
sheet1_df.head(), sheet2_df.head()

In [ ]:
# Limpiar espacios adicionales y convertir a minúsculas en ambas hojas
sheet2_df['NOMBRES Y APELLIDOS'] = sheet2_df['NOMBRES Y APELLIDOS'].str.lower().str.strip()
sheet1_df['APELLIDOS Y NOMBRES'] = sheet1_df['APELLIDOS Y NOMBRES'].str.lower().str.strip()

# Reemplazar "v" por "b" en ambas hojas para evitar errores tipográficos comunes
sheet1_df['APELLIDOS Y NOMBRES'] = sheet1_df['APELLIDOS Y NOMBRES'].str.replace('v', 'b')
sheet2_df['NOMBRES Y APELLIDOS'] = sheet2_df['NOMBRES Y APELLIDOS'].str.replace('v', 'b')

# Mostrar una muestra de los datos normalizados
sheet1_df[['APELLIDOS Y NOMBRES']].head(), sheet2_df[['NOMBRES Y APELLIDOS']].head()

# Función para calcular la similitud entre dos cadenas
def calcular_similitud(nombre1, nombre2):
    # Asegurarse de que ambos argumentos sean cadenas de texto
    if not isinstance(nombre1, str):
        nombre1 = str(nombre1)
    if not isinstance(nombre2, str):
        nombre2 = str(nombre2)
    
    return SequenceMatcher(None, nombre1, nombre2).ratio()

# Comparar los nombres de ambas hojas
resultados = []
threshold = 0.75  # Establecer un umbral de similitud del 80%

# Iterar sobre los nombres de la hoja 2 (solicitud)
for nombre2 in sheet2_df['NOMBRES Y APELLIDOS']:
    encontrado = False
    # Comparar con todos los nombres de la hoja 1 (base)
    for nombre1 in sheet1_df['APELLIDOS Y NOMBRES']:
        similitud = calcular_similitud(nombre1, nombre2)
        if similitud >= threshold:
            encontrado = True
            break
    resultados.append('Encontrado' if encontrado else 'No encontrado')

# Agregar la columna de resultados (Estado) al archivo de solicitud
sheet2_df['Estado'] = resultados

# Guardar el archivo actualizado con la columna de estado
output_path = 'Resultados.xlsx'
sheet2_df.to_excel(output_path, index=False)

# Devolver la ruta del archivo generado
output_path